# JEC profile plots

Simple mix of https://github.com/cms-jet/CoffeaJERC/blob/master/genL2L3.ipynb and [nanoevents.ipynb](https://github.com/CoffeaTeam/coffea/blob/master/binder/nanoevents.ipynb) to illustrate profile plots

In [ ]:
import awkward as ak
import numpy as np
import time
from coffea.nanoevents.methods import candidate
from coffea.nanoevents import NanoEventsFactory, NanoAODSchema

In [ ]:
ak.behavior.update(candidate.behavior)

### Decide whether to load filesets from a json file or enter individual root files
Set `LoadJSONfiles` to `True` to import filesets with several root files from a given .json file.

In [ ]:
LoadJSONfiles = True

### Change to your xrootd username, unless you're AC Williams ;)

In [ ]:
xrootdstr = 'root://acwillia@cmsxrootd.fnal.gov/'

In [ ]:
filesets = {}

if not LoadJSONfiles:
    fname = xrootdstr + "/store/mc/RunIISummer19UL17NanoAOD/QCD_Pt-15to7000_TuneCP5_Flat2018_13TeV_pythia8/NANOAODSIM/JMECustomTuples_106X_mc2017_realistic_v6-v1/280000/0F7E67F1-5FCB-EC4B-A0B3-E0E9B98AFC43.root"
    #events = NanoEventsFactory.from_root(fname, schemaclass=NanoAODSchema).events()
    
    filesets = {
        "QCDFlat": [fname]
    }
    
else:
    import json
    json_samples = json.load( open('samples_qcdflat.json') )
    
    for sample in json_samples["samples"]:    
        name, xsec, nevents, files = sample['name'], sample['xsec'], sample['nevents'], sample['files']
        for ifile,file in enumerate(files):
            files[ifile] = xrootdstr + file
        filesets[name] = files

### Define the processor, or import one from a separate `.py` file

In [ ]:
from coffea import processor, hist
class FancyJECL2L3Processor(processor.ProcessorABC):
    def __init__(self):
        dataset_axis = hist.Cat("dataset", "Primary dataset")
        eta_axis = hist.Bin("eta", r"$\eta$", 20, -5, 5)
        pt_axis = hist.Bin("pt", r"$p_{T}$ [GeV]", 
                           np.array([5,10,15,20,25,30,35,40,45,50,60,70,80,90,
                                     100,120,140,160,180,
                                     200,250,300,350,400,450,500,
                                     600,700,800,900,1000,
                                     1500,2000,3000,4000,5000]))
        dr_axis = hist.Bin("dr", r"$\delta (\eta)$", 20, 0., 1)
        m_axis = hist.Bin("m", r"$p_{T}$ [GeV]", 200, 0, 500)
        r_axis = hist.Bin("r", "RECO / GEN response", 200, 0, 5)
        
        self._accumulator = processor.dict_accumulator({
            'pt':hist.Hist("Counts", dataset_axis, pt_axis),
            'eta':hist.Hist("Counts", dataset_axis, eta_axis),
            'dr':hist.Hist("Counts", dataset_axis, dr_axis),
            'r_pt_ptveta':hist.Hist("Counts", dataset_axis, pt_axis, eta_axis, r_axis),
            'r_m_ptveta':hist.Hist("Counts", dataset_axis, pt_axis, eta_axis, r_axis),
            'r_m_ptvm':hist.Hist("Counts", dataset_axis, pt_axis, m_axis, r_axis),
            'cutflow': processor.defaultdict_accumulator(int),
        })
    
    @property
    def accumulator(self):
        return self._accumulator
    
    def process(self, events):
        output = self.accumulator.identity()
        output['cutflow']['all events'] += len(events)
        #print(len(events))
            
        selectedEvents = events[
            (ak.num(events.Jet) > 2)
        ]

        jet = selectedEvents.Jet[:,0:2]
        jet = ak.flatten(jet)
        
        # --- only with genmatch --- #
        jet = jet[~ak.is_none(jet.matched_gen)]
        
        # --- only with good deltaR match --- #
        jet = jet[jet.delta_r(jet.matched_gen)<0.2]
        
        ptresponse = jet.pt/jet.matched_gen.pt
        
        output['dr'].fill(dataset=selectedEvents.metadata["dataset"],
                            dr=jet.delta_r(jet.matched_gen))
        output['pt'].fill(dataset=selectedEvents.metadata["dataset"],
                            pt=jet.pt)
        output['eta'].fill(dataset=selectedEvents.metadata["dataset"], 
                                 eta=jet.eta)
        output['r_pt_ptveta'].fill( dataset=selectedEvents.metadata["dataset"], pt=jet.pt, eta=jet.eta, r=ptresponse)
        
        return output

    def postprocess(self, accumulator):
        return accumulator
   

### Try the experimental dask processor

If you'd like ... 

In [ ]:
UsingDaskExecutor = False

In [ ]:
if UsingDaskExecutor == True:
    from dask.distributed import Client
    from lpcjobqueue import LPCCondorCluster
    if __name__ == "__main__":
        tic = time.time()
        cluster = LPCCondorCluster()
        # minimum > 0: https://github.com/CoffeaTeam/coffea/issues/465
        cluster.adapt(minimum=1, maximum=10)
        client = Client(cluster)
        #client.upload_file('FancyJECL2L3Processor.py')
        
    tstart = time.time() 
    output = processor.run_uproot_job(
        filesets,
        treename           = "Events",
        processor_instance = FancyJECL2L3Processor(),
        executor           = processor.dask_executor,
        executor_args      = {"client": client, 
                              "schema": NanoAODSchema},
    )
    elapsed = time.time() - tstart
    print(output)
    print("Total time from dask: ", elapsed)
    print()
    print("Events/s:", output['cutflow']['all events']/elapsed)
    
else: #Just use iterative executor
    tstart = time.time() 
    output = processor.run_uproot_job(
        filesets,
        treename           = "Events",
        processor_instance = FancyJECL2L3Processor(),
        executor           = processor.iterative_executor,
        executor_args      = {"schema": NanoAODSchema},
    )
    elapsed = time.time() - tstart
    print(output)
    print("Total time from iterative executor: ", elapsed)
    print()
    print("Events/s: ", output['cutflow']['all events']/elapsed)

In [ ]:
import mplhep as hep
import matplotlib.pyplot as plt
import scipy.optimize as sciop
import scipy.special as scisp 
#import seaborn as sb

### Define the Fit Function(s)

In [ ]:
def GammaFit(x, alpha, beta, amp):
    return ( amp * beta**(alpha) * x**(alpha-1) * np.exp(-beta*x) / scisp.gamma(alpha) )

def GaussianFit1(x, xbar, A, sigma):
    return ( A * np.exp(-0.5*(x - xbar)**2/sigma**2) )

def GaussianFit2(x, xbar, sigma):
    return ( (1./sigma/np.sqrt(2*np.pi)) * np.exp(-0.5*(x - xbar)**2/sigma**2) )

Define stuff thats gonna be used over and over again.  Check and make sure this stuff gives what I think it gives.  For `PtResponseCounts`, remember that its two dimensional; different counts for each eta bin and pt bin.

In [ ]:
EtaBins = output['r_pt_ptveta'].axis('eta')
EtaBinNums = len(output['eta'].values()[('QCDFlat',)])
EtaCounts = output['eta'].values()[('QCDFlat',)]

PtBins = output['r_pt_ptveta'].axis('pt')
PtBinNums = len(output['pt'].values()[('QCDFlat',)])
PtCounts = output['pt'].values()[('QCDFlat',)]

PtResponseBins = output['r_pt_ptveta'].axis('r')
PtResponseBinNums = len(output['r_pt_ptveta'].integrate('eta', EtaBins[1]).integrate('pt',PtBins[1]).values()[('QCDFlat',)])
def PtResponseCounts(etabin_number, ptbin_number): # takes integer values of bin numbers
    Counts = output['r_pt_ptveta'].integrate('eta', EtaBins[etabin_number]).integrate('pt',PtBins[ptbin_number]).values()[('QCDFlat',)]
    return Counts

# --- yaxis values of histograms for each bin --- #
print('# of Eta Bins = ', EtaBinNums)
print('# of pT Bins = ', PtBinNums)
print('# of Response Bins = ', PtResponseBinNums)
print()
print('Eta Counts: ', EtaCounts)
print('\npT Counts: ', PtCounts)
print('\nResponse Counts:', PtResponseCounts(7,15))

### List all Bins and Bin Numbers for Future Reference if Needed

In [ ]:
ax = hist.plotgrid(output['eta'], overlay="dataset", stack=False, density=True)
for i in range(EtaBinNums):
    print('Bin #' + str(i) + ': '+ str(EtaBins[i]))

In [ ]:
ax = hist.plotgrid(output['pt'], overlay="dataset", stack=False, density=True)
for i in range(PtBinNums):
    print('Bin #' + str(i) + ': '+ str(PtBins[i]))

In [ ]:
for i in range(PtResponseBinNums):
    print('Bin #' + str(i) + ': '+ str(PtResponseBins[i]))

# $p_T$ Response for select eta and pt ranges

In [ ]:
collect_etaranges = [] # for violin plots later...
for bins in EtaBins[1:20]:
    choicebin = PtBins[15] # choose a single pt bin
    Hists = output['r_pt_ptveta'].sum('dataset').integrate('eta', bins).integrate('pt',choicebin)
    collect_etaranges.append(list(Hists.values().values())[0])
    title = r'$\eta$ range ' + str(bins) + r'; $p_T$ range ' + str(choicebin)
    ax = hist.plot1d(Hists)
    ax.set_title(title)
    plt.show()

In [ ]:
collect_ptranges = [] # for violin plots later...
for bins in PtBins[2:35]:
    choicebin = EtaBins[11] # choose a single eta bin
    Hists = output['r_pt_ptveta'].sum('dataset').integrate('eta', choicebin).integrate('pt',bins)
    collect_ptranges.append(list(Hists.values().values())[0])
    title = r'$\eta$ range ' + str(choicebin) + r'; $p_T$ range ' + str(bins)
    ax = hist.plot1d(Hists)
    ax.set_title(title)
    plt.show()

# Loop Over All $p_T$ Responses and Fit the Curves

In [ ]:
PtResponseBins_array = np.arange(0, 5, 0.025) # PtResponseBins should be an array for the xdata of curve_fit
xspace = np.linspace(0, 5, 100000) # Generate more x values if curve is not smooth enough
xbar_vals = [] # Store mean values for each eta bin (outer) and pt bin (inner)
sigma_vals = [] # Store standard deviation values for each eta bin (outer) and pt bin (inner)

for i in range(EtaBinNums):
    inner_xbar_array = []
    inner_sigma_array = []
    for j in range(PtBinNums):
        if not (i>0) or not (j>1): # Skip first eta bin, first pt bin and second pt bin to avoid empty histograms
            continue
        elif j == 35: # Skip last pt bin to avoid empty histograms
            continue
        else:
            Hists = output['r_pt_ptveta'].sum('dataset').integrate('eta', EtaBins[i]).integrate('pt',PtBins[j])
        
            # --- fit --- #
            try:
                popt1, pcov1 = sciop.curve_fit(GaussianFit1, xdata=PtResponseBins_array, ydata=PtResponseCounts(i, j), p0=[1., 1., 0.1], bounds=([0.5, 1.0, 0.01], [1.5, 500., 1]))
                print('Guassian1 fit param. (mean, amp, stddev) = ', popt1)
                xbar, sigma = popt1[0], popt1[2]
                inner_xbar_array.append(xbar)
                inner_sigma_array.append(sigma)

            except RuntimeError:
                print('Optimal parameters not found; eta bin ' + str(EtaBins[i]) + '; pt bin ' + str(PtBins[j]) + ' skipped')
                inner_xbar_array.append(0)
                inner_sigma_array.append(0)
                continue

            title = r'$\eta$ range ' + str(EtaBins[i]) + r'; $p_T$ range ' + str(PtBins[j])
            ax = hist.plot1d(Hists)
            ax.set_title(title)

            plt.plot(PtResponseBins_array, GaussianFit1(PtResponseBins_array, *popt1), color='red', linewidth=2.5, label=r'Gaussian 1')
            plt.show()
        
    xbar_vals.append(inner_xbar_array)
    sigma_vals.append(inner_sigma_array)

In [ ]:
np.array(xbar_vals)
np.array(sigma_vals)

# 2D $p_T$ Response Plots

In [ ]:
for bins in EtaBins:
    title = r'$\eta$ range ' + str(bins)
    ax = hist.plot2d(output['r_pt_ptveta'].sum('dataset').integrate('eta', bins),xaxis='pt')
    ax.set_title(title)

In [ ]:
for bins in PtBins:
    title = r'$p_T$ range ' + str(bins)
    ax = hist.plot2d(output['r_pt_ptveta'].sum('dataset').integrate('pt', bins),xaxis='eta')
    ax.set_title(title)

In [ ]:
for bins in EtaBins:
    h = output['r_pt_ptveta'].sum('dataset').integrate('eta', bins)
    xaxis='pt'
    xaxis = h.axis(xaxis)
    yaxis = h.axes()[1]
    xoverflow='none'
    xedges = xaxis.edges(overflow=xoverflow)
    xcenters = xaxis.centers(overflow=xoverflow)
    vals = list(h.values().values())

    avs = [np.average(h.axes()[1].centers(), weights=b) if np.sum(b)>0 else 0. for b in vals[0]]
    #dummy
    avs_err = [0.]*len(avs)

    fig, ax = plt.subplots(1, 1)

    ax.set_xlabel(xaxis.label)
    ax.set_ylabel(yaxis.label)
    ax.set_xlim(xedges[0], xedges[-1])
    ax.set_ylim(0.5, 1.5)

    errbar = ax.errorbar(x=xcenters, y=avs, yerr=avs_err)
    
    title = title = r'$\eta$ range ' + str(bins)
    ax.set_title(title)
    plt.xscale("log")

In [ ]:
for bins in PtBins:
    h = output['r_pt_ptveta'].sum('dataset').integrate('pt', bins)
    xaxis='eta'
    xaxis = h.axis(xaxis)
    yaxis = h.axes()[1]
    xoverflow='none'
    xedges = xaxis.edges(overflow=xoverflow)
    xcenters = xaxis.centers(overflow=xoverflow)
    vals = list(h.values().values())

    avs = [np.average(h.axes()[1].centers(), weights=b) if np.sum(b)>0 else 0. for b in vals[0]]
    #dummy
    avs_err = [0.]*len(avs)

    fig, ax = plt.subplots(1, 1)

    ax.set_xlabel(xaxis.label)
    ax.set_ylabel(yaxis.label)
    ax.set_xlim(xedges[0], xedges[-1])
    ax.set_ylim(0.5, 1.5)

    errbar = ax.errorbar(x=xcenters, y=avs, yerr=avs_err)
    
    title = title = r'$p_T$ range ' + str(bins)
    ax.set_title(title)
    #plt.xscale("log")

Would be awesome to have a kind of projection function that gives "profile plots", e.g. 
* showing arithmetic mean +/- error, 
* median +/- errror (or interquartile range), 
* mode (e.g. from a Gaussian fit)
* violin plots etc.

In [ ]:
collections = [PtCounts, EtaCounts]

In [ ]:
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
ax.violinplot(collections, showmedians=False, showmeans=True, showextrema=False)
ax.set_xlabel('Outputs')
ax.set_ylabel('Counts')
ax.set_xticks([1,2])
ax.set_xticklabels([r'$p_T$', r'$\eta$'])
plt.show()

In [ ]:
'''Smaller range of collections (fewer violin plots at a time) if you want to see the details of the violin'''
a = (collect_ptranges[2], collect_ptranges[3])
b = (collect_etaranges[4], collect_etaranges[5])

In [ ]:
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
ax.violinplot(a, showmedians=True, showmeans=True, showextrema=True)
ax.set_xlabel(r'$p_T$ Bin Number')
ax.set_ylabel('Counts')
plt.show()

In [ ]:
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
ax.violinplot(collect_etaranges, showmedians=True, showmeans=True, showextrema=True)
ax.set_xlabel(r'$\eta$ Bin Number')
ax.set_ylabel('Counts')
plt.show()